In [1]:
# run once (may have to restart notebook)
# !pip install gpt-2-simple
# !pip install tensorflow==1.15

In [1]:
import pandas as pd
import numpy as np
import json
import gpt_2_simple as gpt2
import os
import requests
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

In [3]:
dir_path = "../data/"
dem_file_name = "democrats_result.txt"
sample_dem_name = "democrats_sample.txt"
# if not os.path.exists(dir_path):
#     os.makedirs(dir_path)
# if not os.path.isfile(dir_path + file_name):
#     url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
#     data = requests.get(url)
#     with open(dir_path + file_name, 'w') as f:
#         f.write(data.text)

In [4]:
# with open(dir_path + dem_file_name,'r') as f:
#     data = f.readlines(10000)
# with open(dir_path + "democrats_sample.txt",'w+') as to_write:
#     to_write.writelines(data)
# # data

In [5]:
start = datetime.now()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dir_path + sample_dem_name,
              model_name=model_name,
              print_every=20,
              steps=200)   # steps is max number of training steps

gpt2.generate(sess)
end = datetime.now()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/run1/model-3
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-3


100%|██████████| 1/1 [00:00<00:00, 183.35it/s]

Loading dataset...
dataset has 2348 tokens
Training...


[20 | 2079.49] loss=0.20 avg=0.20
[40 | 4121.58] loss=0.02 avg=0.11
[60 | 6144.13] loss=0.02 avg=0.08
[80 | 8161.42] loss=0.02 avg=0.06
[100 | 10213.83] loss=0.01 avg=0.05
======== SAMPLE 1 ========
? Yes i hump my guns And get rich Selling guns to the public at large is selfish and would do us in if we was a human being. It's not about score a few extra ops rolls or whatever it is. This is america. Awesome.

Then along came that idiot tweet who had this exact same problem with chan months later. He had the audacity to link to this post if it wasn't confirmed already. He also had the gall to put up with racist language for over a year. I'm not blaming anyone, just proving that he is an idiot. I blame myself for contributing to his decision. blame everyone in a haze of negativity. What caused all of this? I just wrote this into an election so as to vote. I don't have a lick of policy experience. I don. Joe. Biden. congress do. have experience with a thesaurus link getting linked to a li

In [6]:
end-start

datetime.timedelta(seconds=20564, microseconds=578251)

In [7]:
gpt2.generate(sess, temperature=.7)

Examining the story of Alex Jones is like looking at a photo of a trophy having a good time. It's not an accident that he has links to extremist websites. He is the person having a field day. He is the people being authoritarian. And they clearly need mental health being who our president is lol What a pathetic lot you all are. Trump is pretty much the [Supply side Jesus](https://www.youtube.com/watch?v=X8xU-gKK17A). No democrats want to take your guns. Many democrats own guns, especially in my home state of Wisconsin. 


We are not the ones being authoritarian, that would be the guy who teargassed and flashbanged members of a church in DC yesterday so he could get a photo op of himself standing outside holding a bible upside down. This is America. It's not a new problem. Since our country was founded, we've been divided by race. We've systematically taken away rights of people who aren't like us. We've had authoritarians running ostensibly democratic governments to perpetuate racism.


In [9]:
gpt2.generate(sess, temperature=1.0)

This article is a disservice to both the average Joe and the Republican Party. It also casts doubt on the legitimacy of 2016 even as we work to strengthen it. If we stop at just one thing. Island Townships. There is always going to be violence and dishonesty within the world. But if we stop at taxing and regulating and making money. Then there's always going to be irresponsibility on your part. You are the real hero. And us. But also us who have the power to make them happen.

We can only hope we come out of this with a rejuvenated sense of racial equality. And fight for that we must. For context, my parents are Indian. My ancestors grew up in slums, earning 1000 dollars a year or less, living in shacks, working meal to meal, wearing rags. My grandpa had to slog his ass off nearly working himself to death to work his way up in a factory with dismal conditions and finally take ownership. Why? So he could send my dad to high shcool and so he could come to college here. 20 years later and

In [11]:
gpt2.generate(sess, temperature=1.5)

Baby Boomers can now easily imagine been an approval connector and boss's staunch opposition to gun restrictions. Showing alarm over the many courses the Tea Party have taken is a recurring theme. Feckless. If we solve social      NGOS is anybody's patriots first republic. They are the ones behind the f--- up DC. And their mad partnerships w/wisconsin and somebody may128 right Trumpet there. There's a republican subreddit including republican.com If anyone knows how Japanese visit the <a\https://www.reddit.com/r/The_Donald/>core<a/> of the <a\https://www.facebook.com/realDonaldTrump/posts/1217648249754288? VOTE there.<b>https://www.visualcapitalist.com/ranked-the-fundraising-assistance/ ‎... other republican posts include: Cultural Marxism: From Nice To Ricardous

Karl Rove's Big Oil Party

Reaganmania

Astrology

******

The into which we draw the line

David

******

The people that have the power to make these changes are elected officials. That means the people that have the power 

In [12]:
gpt2.generate(sess, temperature=1.6)

The Caps could use a boost from the this vote and gun guy even if it's shining a spotlight oning Dem support. Just pointing out identifiers. Just obeying the process. There's always going to be violence and dishonesty within the world. But when it comes to mental health being consistent with whats was said in the 1st election, we judges by the health we have in power. From having approved 95 regulations at the time of the poll pouring rain on us but also lacking the will to act. Cuccinelli *is Senator Cruz's sleep target. They'll proof vote your butts off when you teach kids he was never elected President. He opted out becult he wasn't disciplined. You and i just act like youhatewhomctratsstckedboughtem/rumorsinfo if youhe beaten usGRDC by predictingagame blowback. And guess how much longer does it take usu treat grant bums like miscreants123 why does his facebook back ban work? why is there the unconstitutional practice thereof

reports- esconfirmed on sockeye. prospa ls tradition ofh